Let $f \in \mathbb{Z}[x]$ be a monic polynomial of degree $n$, which we assume has no repeated roots. The Galois group of $f$ is $Gal(f) = Gal(K/\mathbb{Q})$, where $K$ is the splitting field of $f$. It acts by permuting the roots $\alpha_1, \dots, \alpha_n$ of $f$, and hence is a subgroup of $S_n$.

Now let $S_n$ act on the multivariate polynomial ring $P = \mathbb{Z}[X_1, \dots, X_n]$ by permuting the indeterminates $X_1, \dots, X_n$. Let $h_1, \dots, h_m$ be the orbit of some multivariate polynomial $h \in P$ (with say $h_1 = h$) under this action and write $Stab(h) \leq S_n$ for the stabiliser. The resolvent of $f$ with respect to $h$ is the polynomial
\begin{equation}
    R_h(f) = \prod_{i=1}^m (x − h_i(\alpha_1, \dots, \alpha_nn)).
\end{equation}
If $R_h(f)$ has distinct roots, then it can be shown that $Gal(f)$ is conjugate in $S_n$ to a subgroup of $Stab(h)$ if and only if $R_h(f)$ has an integer root.

Take $h(X_1, \dots, X_n) = X_1$. Applying a permutation $\sigma \in S_n$ to $h = X_1$ gives $\sigma(h) = X_{\sigma(1)}$ where $\sigma(1)$ can be any index from $1$ to $n$. Therefore, the orbit of $h$ is the set $\{X_1, \dots, X_n\}$. The roots of the resolvent are obtained by evaluating the elements of the orbit at the roots of $f$ which are simply the roots of $f$ itself. Since $f$ is monic, the resolvent polynomial is the original polynomial $f$,
\begin{equation}
    R_h(f) = (x - \alpha_1) \cdots (x - \alpha_n) = f(x).
\end{equation}
The stabiliser is the set of permutations $\sigma \in S_n$ that leave $h = X_1$ unchanged but can permute the other indeterminates arbitrarily, i.e., $Stab(h) \cong S_{n-1}$. Hence, a Galois group is a subgroup of $S_{n-1}$ if and only if its action on the roots fixes at least one root. A root is fixed by the entire Galois group if and only if it belongs to the base field $\mathbb{Q}$ and therefore $\mathbb{Z}$ by the rational root theorem.

Now take $h(X_1, \dots, X_n) = \prod_{i<j} (X_i - X_j)$. When a permutation $\sigma \in S_n$ acts on $h$, it reorders the terms. The effect is that $h$ is multiplied by $\mathop{sgn}(\sigma) = \pm 1$, so $\sigma(h) = \pm h$. Therefore, the orbit of $h$ is the set $\{-h, h\}$. We have
\begin{equation}
    h(\alpha_1, \dots, \alpha_n) = \prod_{i<j} (\alpha_i - \alpha_k) = \sqrt{\Delta(f)}.
\end{equation}
The roots of the resolvent are $\pm\sqrt{\Delta(f)}$, so the resolvent polynomial is
\begin{equation}
    R_h(f) = x^2 - \Delta(f).
\end{equation}
The stabiliser is the set of even permutations which is, of course, the alternating group $Stab(h) \cong A_n$. Then $R_h(f)$ has an integer root is equivalent to saying the discriminant is a perfect square $\Delta(f) = k^2$. This tells us that the Galois group $Gal(f)$ is a subgroup of the alternating group $A_n$ if and only if the discriminant $\Delta(f)$ is a perfect square.


---

The resolvent is defined as $R_h(f) = \prod_i (x - \beta_i)$, where the $\beta_i$ are the values obtained by evaluating the polynomials in the orbit of $h$ under the action of $S_n$ at the roots of $f$. Since the coefficients of the resolvent are symmetric functions of the roots of $f$, they must be integers. This allows us to use floating-point approximations to find the roots $\beta_i$ and then round the resulting coefficients of $R_h(f)$ to the nearest integer.

*   Compute the roots of $f$, denoted $\alpha_1, \dots, \alpha_n$. These roots will be floating-point complex numbers.
*   Generate all $n!$ permutations $(\alpha_{\sigma(1)}, \dots, \alpha_{\sigma(n)})$ of the roots of $f$ and evaluate $h$.
*   Filter to a set of unique values $\{\beta_1, \dots, \beta_m\}$ with a small tolerance.
*   Construct the resolvent polynomial from its roots $\{\beta_1, \dots \beta_m\}$ using rounded coefficients.

In [62]:
import numpy as np
from itertools import permutations

def get_unique_roots(root_list, tolerance=1e-9):
    '''
    Filters a list of complex numbers to find unique values within a tolerance.
    '''
    if not root_list:
        return []

    # Sort complex numbers first by real part, then by imaginary part
    root_list.sort(key=lambda c: (c.real, c.imag))

    unique_roots = [root_list[0]]
    for i in range(1, len(root_list)):
        if abs(root_list[i] - root_list[i-1]) > tolerance:
            unique_roots.append(root_list[i])

    return unique_roots

def compute_resolvent(f_coeffs, h_func):
    '''
    Computes the resolvent polynomial R_h(f) for a monic polynomial f
    with integer coefficients.
    Args:
        f_coeffs: A list of integer coefficients for the monic polynomial f.
        h_func: A function that takes a list of n roots (the alpha_i) and
                computes the value of the multivariate polynomial h.

    Returns:
        A list of integer coefficients for the resolvent polynomial R_h(f).
    '''
    n = len(f_coeffs) - 1

    # Find the roots of f numerically
    roots_alpha = np.roots(f_coeffs)

    # Generate all permutations of the indices of the roots
    indices = range(n)
    all_perms_of_indices = permutations(indices)

    # Evaluate h for each permutation of the roots
    resolvent_root_values = []
    for p in all_perms_of_indices:
        permuted_roots = [roots_alpha[i] for i in p]
        value = h_func(permuted_roots)
        resolvent_root_values.append(value)

    # Find the unique roots of the resolvent polynomial
    unique_resolvent_roots = get_unique_roots(resolvent_root_values)

    # Construct the resolvent polynomial from its roots
    resolvent_coeffs_float = np.poly(unique_resolvent_roots)

    # Round the floating-point coefficients to the nearest integers
    resolvent_coeffs_int = [int(round(c.real)) for c in resolvent_coeffs_float]

    return resolvent_coeffs_int

# Case 1: n=4, h = X₁X₂ + X₃X₄
def h1_n4(roots):
    '''
    Takes a list of 4 roots [a1, a2, a3, a4].
    '''
    return roots[0] * roots[1] + roots[2] * roots[3]

# Case 2: n=4, h = X₁X₂² + X₂X₃² + X₃X₄² + X₄X₁²
def h2_n4(roots):
    '''
    Takes a list of 4 roots [a1, a2, a3, a4].
    '''
    a = roots
    return a[0]*a[1]**2 + a[1]*a[2]**2 + a[2]*a[3]**2 + a[3]*a[0]**2

# Case 3: n=5, h = Σ_{i=1 to 5} Xᵢ²(X_{i+1}X_{i+4} + X_{i+2}X_{i+3})
def h3_n5(roots):
    '''
    Takes a list of 5 roots. Indices are read mod 5.
    '''
    a = roots
    total = 0
    for i in range(5):
        i1 = (i + 1) % 5
        i4 = (i + 4) % 5
        i2 = (i + 2) % 5
        i3 = (i + 3) % 5
        term = a[i]**2 * (a[i1]*a[i4] + a[i2]*a[i3])
        total += term
    return total


f_quartic = [1, 0, 0, -1, 1]
print(f"Testing with f(x) = x^4 - x + 1 (coeffs: {f_quartic})")
resolvent1 = compute_resolvent(f_quartic, h1_n4)
print(f"The resolvent polynomial has coefficients: {resolvent1}")
resolvent2 = compute_resolvent(f_quartic, h2_n4)
print(f"The resolvent polynomial has coefficients: {resolvent2}")

f_quintic = [1, 0, 0, 0, -1, 1]
print(f"\nTesting with f(x) = x^5 - x + 1 (coeffs: {f_quintic})")
resolvent3 = compute_resolvent(f_quintic, h3_n5)
print(f"The resolvent polynomial has coefficients: {resolvent3}")

Testing with f(x) = x^4 - x + 1 (coeffs: [1, 0, 0, -1, 1])
The resolvent polynomial has coefficients: [1, 0, -4, -1]
The resolvent polynomial has coefficients: [1, 12, 100, 559, 2401, 7901, 19710, 36765, 40683, 10458, -86194, -146847, -168163]

Testing with f(x) = x^5 - x + 1 (coeffs: [1, 0, 0, 0, -1, 1])
The resolvent polynomial has coefficients: [1, -2, 46, -110, 1152, -7826, 8529, -186892, -13730, -3114163, 3043808, 2428169, 99865875]


For polynomials that are irreducible over $\mathbb{Q}$, since $\mathbb{Q}$ has chracteristic $0$, we can assume that each polynomial is guaranteed to have distinct roots. Indeed, if $f(x)$ and $f'(x)$ have a common root, then $\gcd(f, f')$, is a polynomial of degree at least $1$. However, since $f(x)$ is irreducible, its only divisors are constants and associates of $f(x)$ itself, so $\gcd(f, f')$ is either constant or a scalar multiple of $f$. This implies that f(x) must divide f'(x) but $\deg(f') + 1 = \deg(f)$ by characteristic zero, which means that $f(x)$ is the zero polynomial, giving a contradiction.

To investigate the Galois groups, we will use the discriminant and resolvent polynomials.
*   We compute $\Delta(f) = (-1)^{n(n-1)/2}\mathop{Res}(f, f')$. If $\Delta$ is a perfect square of an integer, then $Gal(f)$ is a subgroup of the alternating group $A_n$. Otherwise, it is not.
*   We use a resolvent polynomial $R_h(f)$ corresponding to a specific subgroup Stab(h). If $R_h(f)$ has an integer root, then $Gal(f)$ is a subgroup of $Stab(h)$.
    *   For quartics, we use the resolvent for $h_1 = X_1X_2 + X_3X_4$, whose stabiliser is the dihedral group $D_4$. We also use the resolvant for $h_2 = X_1X_2^2 + X_2X_3^2 + X_3X_4^2 + X_4X_1^2$ whose stabiliser is $C_4$.
    *   For quintics, we use the resolvent for
        \begin{equation}
            h_3 = \sum_{i=1}^5 X_i^2(X_{i+1}X_{i+4} + X_{i+2}X_{i+3}),
        \end{equation}

        where the indices are read modulo $5$ and whose stabiliser is the general affine group $GA(1, 5)$.

We know that the Galois group of an irreducible quartic $f$ must be a transitive subgroup of $S_n$.
*   Discriminant test: $\Delta$ is a perfect square if and only if $Gal(f)$ is a subgroup of $A_4$.
*   Cubic resolvant test for $h_1$: The resolvant has an integer root if and only if $Gal(f)$ is a subgroup of $D_4$.
*   Cubic resolvant test for $h_2$: The resolvant has an integer root if and only if $Gal(f)$ is a subgroup of $C_4$.

1.  $\Delta$ not square, $R_{h_1}(f)$ has no integer root, $Gal(f) = S_4$.
2.  $\Delta$ is square, $R_{h_1}(f)$ has no integer root, $Gal(f) = A_4$.
3.  $\Delta$ not square, $R_{h_1}(f)$ has an integer root, $Gal(f) = D_4 \text{ or } C_4$.
    *   $R_{h_2}(f)$ has an integer root, $Gal(f) = C_4$.
    *   $R_{h_2}(f)$ has no integer root, $Gal(f) = D_4$.
4.  $\Delta$ is square, $R_{h_1}(f)$ has an integer root, $Gal(f) = K_4$.

Similarly, for quintics we can decide whether the Galois group is a subgroup of $A_5$ or $GA(1, 5)$:
1.  $\Delta$ is not a perfect square:
    *   $R_{h_3}(f)$ has no integer root, $Gal(f) = S_5$.
    *   $R_{h_3}(f)$ has an integer root, $Gal(f) = GA(1, 5)$.
2.  $\Delta$ is a perfect square:
    *   $R_{h_3}(f)$ has no integer root, $Gal(f) = A_5$.
    *   $R_{h_3}(f)$ has an integer root, $Gal(f) = D_5 \text{ or } C_5$.

A further test to decide between the last case could be to check for an involution or element of order $2$.

In [63]:
import numpy as np
from itertools import permutations
import math

def get_derivative(f_coeffs):
    n = len(f_coeffs) - 1
    if n <= 0: return [0]
    return [c * (n - i) for i, c in enumerate(f_coeffs[:-1])]

def poly_strip_zeros(coeffs):
    if not coeffs or all(c == 0 for c in coeffs): return [0]
    first_nonzero = next(i for i, c in enumerate(coeffs) if c != 0)
    return coeffs[first_nonzero:]

def pseudo_division(f_coeffs, g_coeffs):
    m, n = len(f_coeffs) - 1, len(g_coeffs) - 1
    if m < n:
        return 1, [0], f_coeffs
    b_n = g_coeffs[0]
    delta = m - n
    r = list(f_coeffs)
    for i in range(delta + 1):
        r = [b_n * c for c in r]
        q_term_coeff = r[0]
        g_mult = [q_term_coeff * c for c in g_coeffs]
        for j in range(len(g_coeffs)):
            r[j] -= g_mult[j]
        r.pop(0)
    c = b_n ** (delta + 1)
    return c, [0], poly_strip_zeros(r if r else [0])

def resultant_integer(f_coeffs, g_coeffs):
    f = poly_strip_zeros(list(f_coeffs))
    g = poly_strip_zeros(list(g_coeffs))
    m, n = len(f) - 1, len(g) - 1
    if n < 0: return 0
    if n == 0: return g[0] ** m if m >= 0 else 1
    if m < n:
        sign = (-1)**(m*n)
        return sign * resultant_integer(g, f)
    c, _, r = pseudo_division(f, g)
    if not r or r == [0]: return 0
    k = len(r) - 1
    b_n = g[0]
    sign = (-1)**(m*n)
    den = c ** (n-k)
    num = sign * (b_n ** (m - k)) * resultant_integer(g, r)
    return num // den

def is_perfect_square(n):
    if n < 0: return False
    if n == 0: return True
    sqrt_n = int(math.sqrt(n))
    return sqrt_n * sqrt_n == n

def evaluate_poly_exact(coeffs, x):
    '''
    Evaluates a polynomial at an integer x using Horner's method.
    '''
    result = 0
    for c in coeffs:
        result = result * x + int(c)
    return result

def has_integer_root(p_coeffs, tolerance=1e-9):
    '''
    A robust method to find integer roots for polynomials with large coefficients.
    It finds numerical approximations and then verifies them with exact arithmetic.
    '''
    if not p_coeffs or all(c == 0 for c in p_coeffs):
        return False # The zero polynomial has no single root.

    if p_coeffs[-1] == 0:
        print("      (Verified integer root found: 0)")
        return True

    # Fast approximation using floating-point numbers
    try:
        # Convert to float for np.roots
        float_coeffs = [float(c) for c in p_coeffs]
        roots = np.roots(float_coeffs)
    except (np.linalg.LinAlgError, ValueError):
        print("      (Warning: Numerical root finding failed. Cannot determine integer roots.)")
        return False # Fallback if numerical method fails

    # Exact verification for integer candidates
    for r in roots:
        # A root is a candidate if its imaginary part is tiny
        if abs(r.imag) < tolerance:
            real_part = r.real
            # and its real part is very close to an integer.
            if abs(real_part - round(real_part)) < tolerance:
                candidate = int(round(real_part))

                if evaluate_poly_exact(p_coeffs, candidate) == 0:
                    print(f"      (Verified integer root found: {candidate})")
                    return True

    return False

def distinguish_D4_C4(f_coeffs):
    '''
    This function is called only when we know Gal(f) is D4 or C4.
    '''
    print("    Ambiguous case (D4 or C4). Using C4 resolvent to distinguish...")

    # Use the h function whose stabilizer is C4
    c4_resolvent = compute_resolvent(f_coeffs, h2_n4)
    print(f"    - C4 Resolvent R(y) has coeffs: {c4_resolvent}")

    if has_integer_root(c4_resolvent):
        print("      C4 Resolvent HAS an integer root. Gal(f) is a subgroup of C4.")
        print("  --> Final Conclusion: Gal(f) is C4.")
    else:
        print("      C4 Resolvent does NOT have an integer root. Gal(f) is NOT a subgroup of C4.")
        print("  --> Final Conclusion: Gal(f) is D4.")

def investigate_galois_group(f_coeffs):
    n = len(f_coeffs) - 1
    print(f"\nInvestigating f(x) with coeffs: {f_coeffs}")

    f_prime_coeffs = get_derivative(f_coeffs)
    res = resultant_integer(f_coeffs, f_prime_coeffs)
    sign = (-1)**((n * (n - 1)) // 2)
    discriminant = sign * res
    is_sq = is_perfect_square(discriminant)

    print(f"  - Discriminant Δ = {discriminant}")
    if is_sq: print(f"    Δ is a perfect square. Gal(f) is a subgroup of A{n}.")
    else: print(f"    Δ is NOT a perfect square. Gal(f) is NOT a subgroup of A{n}.")

    if n == 4:
        d4_resolvent = compute_resolvent(f_coeffs, h1_n4)
        print(f"  - D4 Resolvent R(y) has coeffs: {d4_resolvent}")
        has_int_root = has_integer_root(d4_resolvent)
        if has_int_root:
            print("    D4 Resolvent HAS an integer root. Gal(f) is a subgroup of D4.")
            if is_sq:
                print("  --> Conclusion: Gal(f) is K4.")
            else:
                distinguish_D4_C4(f_coeffs)
        else:
            print("    D4 Resolvent does NOT have an integer root. Gal(f) is NOT a subgroup of D4.")
            if is_sq: print("  --> Conclusion: Gal(f) is A4.")
            else: print("  --> Conclusion: Gal(f) is S4.")
    elif n == 5:
        f20_resolvent = compute_resolvent(f_coeffs, h3_n5)
        print(f"  - GA(1, 5) Resolvent R(y) has coeffs: {f20_resolvent}")
        has_f20_root = has_integer_root(f20_resolvent)
        if has_f20_root:
            print("    GA(1, 5) Resolvent HAS an integer root. Gal(f) is a subgroup of GA(1, 5).")
            if is_sq: print("\n  --> Conclusion: Gal(f) is D5 or C5.")
            else: print("\n  --> Conclusion: Gal(f) is GA(1, 5).")
        else:
            print("    GA(1, 5) Resolvent does NOT have an integer root. Gal(f) is NOT a subgroup of GA(1, 5).")
            if is_sq: print("\n  --> Conclusion: Gal(f) is A5.")
            else: print("\n  --> Conclusion: Gal(f) is S5.")

polynomials = [
    # Quartics
    [1, 0, -7, -6, 1],
    [1, -1, 0, 9, 10],
    [1, 2, 23, 22, 6],
    # Quintics
    [1, 0, -1, -7, -1, -3],
    [1, -1, 0, 8, -7, 3],
    [1, -2, 6, -3, -1, 6]
]
for p in polynomials:
    investigate_galois_group(p)


Investigating f(x) with coeffs: [1, 0, -7, -6, 1]
  - Discriminant Δ = 607370801787086970028032
    Δ is NOT a perfect square. Gal(f) is NOT a subgroup of A4.
  - D4 Resolvent R(y) has coeffs: [1, 7, -4, -64]
      (Verified integer root found: -4)
    D4 Resolvent HAS an integer root. Gal(f) is a subgroup of D4.
    Ambiguous case (D4 or C4). Using C4 resolvent to distinguish...
    - C4 Resolvent R(y) has coeffs: [1, 36, 234, -3024, -25875, 31644, 19764]
      C4 Resolvent does NOT have an integer root. Gal(f) is NOT a subgroup of C4.
  --> Final Conclusion: Gal(f) is D4.

Investigating f(x) with coeffs: [1, -1, 0, 9, 10]
  - Discriminant Δ = 25529573222070790223044804608
    Δ is NOT a perfect square. Gal(f) is NOT a subgroup of A4.
  - D4 Resolvent R(y) has coeffs: [1, 0, -49, -91]
    D4 Resolvent does NOT have an integer root. Gal(f) is NOT a subgroup of D4.
  --> Conclusion: Gal(f) is S4.

Investigating f(x) with coeffs: [1, 2, 23, 22, 6]
  - Discriminant Δ = 258468950190702957